In [ ]:
'''
Rewriting of the decode_netout() only using scalar operations and for loops,
so it can be translated line by line to C. Two functions work for netout shape
and mem_tensor shape.
'''

__author__ = "Abarajithan G"
__copyright__ = "Copyright 2019, Final Year Project"
__credits__ = ["Abarajithan G"]
__version__ = "1.0.0"
__maintainer__ = "Abarajthan G"
__email__ = "abarajithan07@gmail.com"
__status__ = "Research"

### Original Netout function

In [1]:
import numpy as np
import pickle
from copy import deepcopy

ANCHORS = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]
NUM_CLASSES = 80
OBJ_THRESHOLD = 0.3
NMS_THRESHOLD = 0.3

def decode_netout(netout, anchors, nb_class, obj_threshold=0.3, nms_threshold=0.3):
    grid_h, grid_w, nb_box = netout.shape[:3]

    boxes = []
    
    # decode the output by the network
    netout[..., 4] = _sigmoid(netout[..., 4])
    
    netout[..., 5:] = netout[..., 4][..., np.newaxis] * \
        _softmax(netout[..., 5:])
    netout[..., 5:] *= netout[..., 5:] > obj_threshold
    
    for row in range(grid_h):
        for col in range(grid_w):
            for b in range(nb_box):
                # from 4th element onwards are confidence and class classes
                classes = netout[row, col, b, 5:]

                if np.sum(classes) > 0:
                    # first 4 elements are x, y, w, and h
                    x, y, w, h = netout[row, col, b, :4]
                    
                    # center position, unit: image width
                    x = (col + _sigmoid(x)) / grid_w
                    # center position, unit: image height
                    y = (row + _sigmoid(y)) / grid_h
                    w = anchors[2 * b + 0] * \
                        np.exp(w) / grid_w  # unit: image width
                    h = anchors[2 * b + 1] * \
                        np.exp(h) / grid_h  # unit: image height
                    confidence = netout[row, col, b, 4]

                    box = BoundBox(x-w/2, y-h/2, x+w/2, y +
                                   h/2, confidence, classes)
                    box.label = np.argmax(classes)
                    box.score = box.classes[box.label]

                    boxes.append(box)
#                     print(x,y,w,h)
    return boxes

def _sigmoid(x):
    return 1. / (1. + np.exp(-x))


def _softmax(x, axis=-1, t=-100.):
    x = x - np.max(x)

    if np.min(x) < t:
        x = x/np.min(x)*t

    e_x = np.exp(x)

    return e_x / e_x.sum(axis, keepdims=True)

class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, c=None, classes=None, label=None, score=None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax

        self.c = c
        self.classes = classes

        self.label = label
        self.score = score

In [2]:
def custom_softmax(x, dims, start_last=0):
    '''
    x should be a 4D array. Softmax is taken WRT to the last axis
    '''
    x_max = 13  # for numerical stability (avoid NANs). better keep it constant for computational efficiency
    soft_sum = 0

    e_test = np.zeros(x.shape).astype(np.float16) # Can declare a multidim array in C
    e_sum  = 0
    e_temp = 0

    for i in range(dims[0]):
        for j in range(dims[1]):
            for k in range(dims[2]):
                e_sum = 0               # NOTE: sum is taken over the innermost axis ONLY.
                for l in range(start_last,dims[3]):
                    e_sum += np.exp(x[i,j,k,l] - x_max)

                for l in range(start_last,dims[3]):
                    e_test[i,j,k,l] = np.exp(x[i,j,k,l] - x_max)/e_sum
                    
    return e_test

def custom_sigmoid(x):
    return 1. / (1. + np.exp(-x))

## LINE BY LINE TRANSLATION

Following code converts line by line into for loops and compares the accuracy. All numpy operations used are scaler operations (act on a single number, not matrix)

In [3]:
grid_h = 8
grid_w = 12
nb_box = 5
others = 85

out = np.load('data/yolo_out_array.npy').reshape((1, grid_h, grid_w, nb_box, others))[0]
netout = deepcopy(out)

In [4]:
netout[..., 4] = _sigmoid(netout[..., 4])
for i in range(grid_h):
    for j in range(grid_w):
        for k in range(nb_box):
            out[i,j,k,4] = _sigmoid(out[i,j,k,4])
            
np.sum(np.abs(out-netout))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: overflow encountered in exp


0.0005555

In [5]:
softmax_netout_5 = _softmax(netout[..., 5:])
soft_out = custom_softmax(out, (grid_h, grid_w, nb_box, others), start_last=5)

np.max(np.abs(soft_out[...,5:]-softmax_netout_5))

0.0008545

In [6]:
netout[..., 5:] = netout[..., 4][..., np.newaxis] * _softmax(netout[..., 5:])
soft_out = custom_softmax(out, (grid_h, grid_w, nb_box, others), start_last=5)

for i in range(grid_h):
    for j in range(grid_w):
        for k in range(nb_box):
            for l in range(5,others):
                out[i,j,k,l] = out[i,j,k,4] * soft_out[i,j,k,l]
                
np.max(np.abs(out-netout))

0.0004883

In [7]:
netout[..., 5:] *= netout[..., 5:] > OBJ_THRESHOLD

for i in range(grid_h):
    for j in range(grid_w):
        for k in range(nb_box):
            for l in range(5,others):
                out[i,j,k,l] = out[i,j,k,l] * (out[i,j,k,l] > OBJ_THRESHOLD) # This can be directly written in C

np.max(np.abs(out-netout))

0.0004883

## OPTIMIZATION

Above code iterates with nested (4-level: i,j,k,l) for loop through the tensor several times. Once to calculate sigmoid, another to calculate softmax (inside custom_softmax function), then to multiply with softmax and finally to perform thresholding.

Below, I have combined everything (including the operations of custom_sigmoid function) into a single pass. Cross checked. It works (Max absolute error is 0.0004883). 

In [8]:
grid_h = 8
grid_w = 12
nb_box = 5
others = 85

out = np.load('data/yolo_out_array.npy').reshape((1, grid_h, grid_w, nb_box, others))[0]

In [9]:
start_last=5
x_max = 13  # for numerical stability (avoid NANs). better keep it constant for computational efficiency
soft_sum = 0
e_sum  = 0
e_temp = 0
e_sum_next = 0
out_4 = 0
tmp = 0

for i in range(grid_h):
    for j in range(grid_w):                
        for k in range(-1, nb_box):
            if k != -1:
                out_4 = _sigmoid(out[i,j,k,4])
                out[i,j,k,4] = out_4
            
            e_sum_next = 0
            for l in range(start_last,others):
                if k != nb_box-1:
                    e_sum_next += np.exp(out[i,j,k+1,l] - x_max)
                
                if k != -1:
                    tmp = out[i,j,k,l]
                    tmp = out_4 * np.exp(tmp - x_max)/e_sum
                    tmp *= tmp > OBJ_THRESHOLD
                    out[i,j,k,l] = tmp
                    
            e_sum = e_sum_next


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: overflow encountered in exp


In [12]:
netout = np.load('data/yolo_out_array.npy').reshape((1, grid_h, grid_w, nb_box, others))[0]
netout[..., 4] = _sigmoid(netout[..., 4])
netout[..., 5:] = netout[..., 4][..., np.newaxis] * _softmax(netout[..., 5:])
netout[..., 5:] *= netout[..., 5:] > OBJ_THRESHOLD

np.max(np.abs(out-netout))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: overflow encountered in exp


0.0004883

In [13]:
grid_h = 8
grid_w = 12
nb_box = 5
others = 85

out = np.load('data/yolo_out_array.npy').reshape((1, grid_h, grid_w, nb_box, others))[0].astype(np.float16)

def netout_for_loop(out):
    start_last=5
    x_max = 13  # for numerical stability (avoid NANs). better keep it constant for computational efficiency
    x_max = np.array(x_max, dtype=np.float16)
    soft_sum = 0
    e_sum  = 0
    e_temp = 0
    e_sum_next = 0
    out_4 = 0
    tmp = 0
    sum_classes = 0

    boxes = []

    for i in range(grid_h):
        for j in range(grid_w):                
            for k in range(-1, nb_box):
                if k != -1:
                    out_4 = _sigmoid(out[i,j,k,4])
                    out[i,j,k,4] = out_4

                e_sum_next = 0
                sum_classes = 0
                label = 0
                score = 0

                for l in range(start_last,others):
                    if k != nb_box-1:
                        e_sum_next += np.exp(out[i,j,k+1,l] - x_max)

                    if k != -1:
                        tmp = out[i,j,k,l]
                        tmp = out_4 * np.exp(tmp - x_max)/e_sum
                        tmp *= tmp > OBJ_THRESHOLD

                        tmp = tmp.astype(np.float16) # Not relevent to C
                        out[i,j,k,l] = tmp # Line can be removed.

                        sum_classes += tmp
                        if tmp > score: # Get max score and corresponding label
                            score = tmp
                            label = l-5


                e_sum = e_sum_next

                if k == -1:
                    continue
                if sum_classes > 0 and score > OBJ_THRESHOLD:
                    x = out[i,j,k,0]
                    y = out[i,j,k,1]
                    w = out[i,j,k,2]
                    h = out[i,j,k,3]

                    x = (j + custom_sigmoid(x))/grid_w
                    y = (i + custom_sigmoid(y))/grid_h
                    w = ANCHORS[2*k + 0] * np.exp(w) / grid_w
                    h = ANCHORS[2*k + 1] * np.exp(h) / grid_h

                    conf = out[i,j,k,4]

                    box = BoundBox(
                        xmin = x-w/2,
                        ymin = y-h/2,
                        xmax = x+w/2,
                        ymax = y+h/2,
                        c = conf,
                        label = label,
                        score = score
                    ) # not including classes

                    boxes.append(box)
                    
        
    ## Include Non-max suppression algorithm here
    # (Among all bboxes which have same class and IOU > NMS_THRESHOLD, remove all but the max confidence)
    # The NMS algorithm they have used seems too complicated. Following looks good.
    # This is best implemented through a linked list as follows:
    #
    # foreach (bbox1):
    #    foreach (bbox2):
    #        if bbox1.label != bbox2.label:
    #            continue
    #        if (iou(bbox1, bbox2) > NMS_THRESHOLD) and (bbox1.c > bbox2.c):
    #            delete(bbox2)
    
    return boxes

boxes = netout_for_loop(out)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: overflow encountered in exp


In [14]:
grid_h = 8
grid_w = 12
nb_box = 5
others = 85

netout = np.load('data/yolo_out_array.npy').reshape((1, grid_h, grid_w, nb_box, others))[0]


boxes_real = decode_netout(netout = netout, 
              anchors = ANCHORS, 
              nb_class=NUM_CLASSES,
              obj_threshold=OBJ_THRESHOLD,
              nms_threshold=NMS_THRESHOLD)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: overflow encountered in exp


In [15]:
len(boxes_real), len(boxes)

for i in range(len(boxes)):
    print(boxes[i].xmin, boxes_real[i].xmin)
    print(boxes[i].ymin, boxes_real[i].ymin)
    print(boxes[i].xmax, boxes_real[i].xmax)
    print(boxes[i].ymax, boxes_real[i].ymax)
    print(boxes[i].label, boxes_real[i].label)
    print(boxes[i].score, boxes_real[i].score)
    print(' ')

0.25126986151608866 0.25126986151608866
0.0005916962475591703 0.0005916962475591703
0.3795593195239012 0.3795593195239012
0.37573448189209047 0.37573448189209047
2 2
0.7837 0.784
 
0.40440627396696105 0.40440627396696105
-0.007673135185744778 -0.007673135185744778
0.5244583589278985 0.5244583589278985
0.4223563130564427 0.4223563130564427
2 2
0.6865 0.6865
 
0.5577878462219703 0.5577878462219703
-0.014901071594461746 -0.014901071594461746
0.7128487666321266 0.7128487666321266
0.4090858082883508 0.4090858082883508
2 2
0.7563 0.756
 
0.17264868060877522 0.17264868060877522
0.3614984839311178 0.3614984839311178
0.46008837299158767 0.46008837299158767
1.0006838123002584 1.0006838123002584
7 7
0.512 0.512
 
0.627531214524021 0.627531214524021
0.5979452049786681 0.5979452049786681
0.9239364171607398 0.9239364171607398
0.9898764073712462 0.9898764073712462
2 2
0.7305 0.7305
 


## WORKING ON MEM_TENSOR

Above code works on netout.shape = (8,12,5,85). It can work on the mem_tensor.shape = (12,5,85,8) with a slight modification.

Note how I have simply switched the index of the tensors without changing anything else in the code (even the order of for loops stay the same!)

In [17]:
grid_w = 12
nb_box = 5
others = 85
grid_h = 8

f = open('data/f16_yolo_out_mem_bytes.bin', 'rb')
b = np.frombuffer(f.read(), dtype=np.float16)
out = b.reshape(12, 5, 85, 8).copy()

In [18]:
start_last=5
x_max = 13  # for numerical stability (avoid NANs). better keep it constant for computational efficiency
soft_sum = 0
e_sum  = 0
e_temp = 0
e_sum_next = 0

for i in range(grid_h):
    for j in range(grid_w):                
        for k in range(-1, nb_box):
            if k != -1:
                out[j,k,4,i] = _sigmoid(out[j,k,4,i])
            
            e_sum_next = 0
            for l in range(start_last,others):
                if k != nb_box-1:
                    e_sum_next += np.exp(out[j,k+1,l,i] - x_max)
                
                if k != -1:
                    out[j,k,l,i] = out[j,k,4,i] * np.exp(out[j,k,l,i] - x_max)/e_sum
                    out[j,k,l,i] *= out[j,k,l,i] > OBJ_THRESHOLD
            e_sum = e_sum_next

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: overflow encountered in exp


In [19]:
out = np.einsum('bcda->abcd',out)
netout = np.load('data/yolo_out_array.npy').reshape((1, grid_h, grid_w, nb_box, others))[0]
netout[..., 4] = _sigmoid(netout[..., 4])
netout[..., 5:] = netout[..., 4][..., np.newaxis] * _softmax(netout[..., 5:])
netout[..., 5:] *= netout[..., 5:] > OBJ_THRESHOLD

np.max(np.abs(out-netout))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: overflow encountered in exp


0.0004883

In [21]:
grid_w = 12
nb_box = 5
others = 85
grid_h = 8

f = open('data/f16_yolo_out_mem_bytes.bin', 'rb')
b = np.frombuffer(f.read(), dtype=np.float16)
out = b.reshape(12, 5, 85, 8).copy()
out.setflags(write = False)

def mem_tensor_for_loop(out):
    start_last=5
    x_max = 13  # for numerical stability (avoid NANs). better keep it constant for computational efficiency
    x_max = np.array(x_max, dtype=np.float16)
    soft_sum = 0
    e_sum  = 0
    e_temp = 0
    e_sum_next = 0
    conf = 0
    tmp = 0
    sum_classes = 0

    boxes = []

    for i in range(grid_h):
        for j in range(grid_w):                
            for k in range(-1, nb_box):
                if k != -1:
                    conf = _sigmoid(out[j,k,4,i]) # <----------

                e_sum_next = 0
                sum_classes = 0
                label = 0
                score = 0

                for l in range(start_last,others):
                    if k != nb_box-1:
                        e_sum_next += np.exp(out[j,k+1,l,i] - x_max)

                    if k != -1:
                        tmp = out[j,k,l,i]
                        tmp = conf * np.exp(tmp - x_max)/e_sum
                        tmp *= tmp > OBJ_THRESHOLD

                        tmp = tmp.astype(np.float16) # Not relevent to C


                        sum_classes += tmp
                        if tmp > score: # Get max score and corresponding label
                            score = tmp
                            label = l-5


                e_sum = e_sum_next

                if k == -1:
                    continue
                if sum_classes > 0 and score > OBJ_THRESHOLD:
                    x = out[j,k,0,i]
                    y = out[j,k,1,i]
                    w = out[j,k,2,i]
                    h = out[j,k,3,i]

                    x = (j + custom_sigmoid(x))/grid_w
                    y = (i + custom_sigmoid(y))/grid_h
                    w = ANCHORS[2*k + 0] * np.exp(w) / grid_w
                    h = ANCHORS[2*k + 1] * np.exp(h) / grid_h

#                     conf = out[j,k,4,i]
#                     conf = out_4

                    box = BoundBox(
                        xmin = x-w/2,
                        ymin = y-h/2,
                        xmax = x+w/2,
                        ymax = y+h/2,
                        c = conf,
                        label = label,
                        score = score
                    ) # not including classes

                    boxes.append(box)
                    
    ## Above code returns all bbox that have confidence above OBJ_THRESHOLD
    
    ## Include Non-max suppression algorithm here
    # (Among all bboxes which have same class and IOU > NMS_THRESHOLD, remove all but the max confidence)
    # The NMS algorithm they have used seems too complicated. Following looks good.
    # This is best implemented through a linked list as follows:
    #
    # foreach (bbox1):
    #    foreach (bbox2):
    #        if bbox1.label != bbox2.label:
    #            continue
    #        if (iou(bbox1, bbox2) > NMS_THRESHOLD) and (bbox1.c > bbox2.c):
    #            delete(bbox2)
    
    return boxes




boxes = mem_tensor_for_loop(out)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: overflow encountered in exp


In [22]:
len(boxes_real), len(boxes)

for i in range(len(boxes)):
    print(boxes[i].xmin, boxes_real[i].xmin)
    print(boxes[i].ymin, boxes_real[i].ymin)
    print(boxes[i].xmax, boxes_real[i].xmax)
    print(boxes[i].ymax, boxes_real[i].ymax)
    print(boxes[i].label, boxes_real[i].label)
    print(boxes[i].score, boxes_real[i].score)
    print(' ')

0.25126986151608866 0.25126986151608866
0.0005916962475591703 0.0005916962475591703
0.3795593195239012 0.3795593195239012
0.37573448189209047 0.37573448189209047
2 2
0.7837 0.784
 
0.40440627396696105 0.40440627396696105
-0.007673135185744778 -0.007673135185744778
0.5244583589278985 0.5244583589278985
0.4223563130564427 0.4223563130564427
2 2
0.6865 0.6865
 
0.5577878462219703 0.5577878462219703
-0.014901071594461746 -0.014901071594461746
0.7128487666321266 0.7128487666321266
0.4090858082883508 0.4090858082883508
2 2
0.7563 0.756
 
0.17264868060877522 0.17264868060877522
0.3614984839311178 0.3614984839311178
0.46008837299158767 0.46008837299158767
1.0006838123002584 1.0006838123002584
7 7
0.512 0.512
 
0.627531214524021 0.627531214524021
0.5979452049786681 0.5979452049786681
0.9239364171607398 0.9239364171607398
0.9898764073712462 0.9898764073712462
2 2
0.7305 0.7305
 
